In [3]:
import pandas as pd
from pathlib import Path
import os

PROJECT_ROOT: Path = Path(os.getcwd()).resolve().parents[0]

parts = [1, 2, 3]


def read_file(part: int | str, root: Path = PROJECT_ROOT):
    path: Path = root / "data" / "raw" / f"private-fda-part-{str(part)}.xlsx"
    if path.exists():
        return pd.read_excel(path)
    else:
        raise ValueError("File not found")


column_name_translation = {
    "جی ال ان توزیع کننده": "distributor_gln",
    "نام توزیع کننده": "distributor_name",
    "جی ال ان داروخانه": "pharmacy_gln",
    "نام داروخانه": "pharmacy_name",
    "تلفن ثابت داروخانه ": "pharmacy_phone",
    "شماره موبایل موسس داروخانه": "pharmacy_founder_mobile",
    "آدرس  داروخانه": "pharmacy_address",
    "نوع فرآورده": "product_type",
    "کد فرآورده": "product_code",
    "نام برند": "brand_name",
    "کد ژنریک": "generic_code",
    "نام ژنریک": "generic_name",
    "سری ساخت": "batch_number",
    "تاریخ رویداد": "event_date",
    "نام مالک فرآورده": "product_owner_name",
    "نام تامین کننده": "supplier_name",
    "تعداد در بسته": "units_per_package",
    "تعداد بسته ارسال شده": "packages_sent",
    "مجموع ارسال شده": "total_units_sent",
    "ارزش ریالی": "monetary_value",
    "استان": "province",
    "شهرستان": "county",
    "شهر": "city",
    "دانشگاه": "university",
}

columns_to_truncate = [
    "city",
    "county",
    "province",
    "university",
    "pharmacy_address",
    "distributor_gln",
    "event_date",
    "product_type",
    "pharmacy_founder_mobile",
    "pharmacy_phone",
    "pharmacy_name",
    "batch_number",
    "pharmacy_gln",
    "distributor_name",
]
dfs_farsi = [read_file(part) for part in parts]
dfs_english = [df.rename(columns=column_name_translation) for df in dfs_farsi]
dfs_truncated = [df.drop(columns=columns_to_truncate) for df in dfs_english]
for df_uniques in dfs_truncated:
    print(f"Dataframe size: {df_uniques['generic_code'].count()}")  # 15, 1066, 1272
    print("Brands covered:")
    print(df_uniques["brand_name"].unique())
    print(df_uniques["generic_name"].unique())
    print("-" * 124)

Dataframe size: 15
Brands covered:
['اکتانیت   پودر لئوفیلیزه برای تهیه محلول تزریقی پرنترال 250 [iU]  [iU]']
['COAGULATION FACTOR VIII (HUMAN PLASMA DERIVED) INJECTION, POWDER, LYOPHILIZED, FOR SOLUTION PARENTERAL 250 [iU]']
----------------------------------------------------------------------------------------------------------------------------
Dataframe size: 1066
Brands covered:
['آیمافیکس   پودر لئوفیلیزه برای تهیه محلول تزریقی پرنترال 500 [iU]  [iU]'
 'هموناین   پودر لئوفیلیزه برای تهیه محلول تزریقی پرنترال 500 [iU]  [iU]'
 'آیمافیکس500   پودر لئوفیلیزه برای تهیه محلول تزریقی پرنترال 500 [iU]  [iU]']
['FACTOR IX INJECTION, POWDER, LYOPHILIZED, FOR SOLUTION PARENTERAL 500 [iU]']
----------------------------------------------------------------------------------------------------------------------------
Dataframe size: 1272
Brands covered:
['اموکلات   پودر لئوفیلیزه برای تهیه محلول تزریقی پرنترال 500 [iU]  [iU]'
 'هماکتین   پودر لئوفیلیزه برای تهیه محلول تزریقی پرنترال 500 [iU]  [

In [4]:
import re
import numpy as np

pattern = r"(\d+)\s*\[iU\]"
dfs_truncated_dict = {
    "First": dfs_truncated[0],
    "Second": dfs_truncated[1],
    "Third": dfs_truncated[2],
}
overall_unit_sent = []
for name, df_uniques in dfs_truncated_dict.items():
    print(f"{name} Dataframe")
    if name == "Second":
        print("Factor IX ignored")
        print("-" * 124)
        continue
    generics = df_uniques["generic_name"].unique()
    for generic in generics:
        match = re.search(pattern, generic)
        if match:
            dosage = int(match.group(1))
            total_unit = df_uniques["total_units_sent"].sum() * dosage
            overall_unit_sent.append(int(total_unit))
            print(f"Total iU sent: {total_unit}")
        else:
            print("No dosage found")
        print("-" * 124)
print(f"Overall iU consumption: {np.array(overall_unit_sent).sum()}")

First Dataframe
Total iU sent: 164000
----------------------------------------------------------------------------------------------------------------------------
Second Dataframe
Factor IX ignored
----------------------------------------------------------------------------------------------------------------------------
Third Dataframe
Total iU sent: 89345500
----------------------------------------------------------------------------------------------------------------------------
Overall iU consumption: 89509500


In [ ]:
# :: Loading Full FDA Statistics 2023
AMARNAMEH_PATH = PROJECT_ROOT / "data" / "raw" / "private-statistics-2023.xlsx"
df_cumulative = pd.read_excel(AMARNAMEH_PATH, sheet_name="Dataset")

In [ ]:
# Processing Full FDA Statistics 2023
column_name_translation = {
    "نام شرکت پخش": "supplier",
    "نام صاحب برند": "brand_owner",
    "نام شرکت تامین کننده": "supplier_company_name",
    "نام شرکت تولید کننده": "manufacturer_company_name",
    "کد IRC محصول": "irc_code",
    "نام لاتین برند": "brand_latin_name",
    "نام برند": "brand_name",
    "نام لاتین فهرست": "index_latin_name",
    "کد ژنریک": "generic_code",
    "نام ژنریک": "generic_name",
    "تعداد فروش (بسته)": "count_of_package_sold",
    "تعداد در بسته": "count_in_package",
    "فروش عددی": "sale_count",
    "فروش ریالی ( مبتنی بر قیمت مصرف کننده)": "sale_amount",
    "OTC": "otc",
    "بیولوژیک": "biologic",
    "تحت لیسانس": "licensed",
    "کد ATC": "atc_code",
    "کشور تولید کننده": "manufacturer_country",
    "ثابت/فهرست": "constant",
    "تولیدی/وارداتی": "imported_manufactured",
    "ثبتی/فوریتی": "emergency_registered",
}
df_cumulative = df_cumulative.rename(columns=column_name_translation)
df_uniques = df_cumulative[
    ["index_latin_name", "brand_latin_name", "generic_name", "generic_code", "irc_code"]
]

df_uniques = df_uniques[df_uniques["generic_name"].str.contains("factor", case=False)]
df_uniques = df_uniques[
    ~df_uniques["generic_name"].str.contains("viia", case=False)
    & ~df_uniques["generic_name"].str.contains("ix", case=False)
    & ~df_uniques["generic_name"].str.contains("xiii", case=False)
]
df_uniques = df_uniques.drop_duplicates(subset="brand_latin_name")

df = df_cumulative[
    [
        "generic_name",
        "brand_latin_name",
        "count_of_package_sold",
        "sale_count",
        "sale_amount",
    ]
]
df = df[df["generic_name"].str.contains("factor viii", case=False)]
agg_dict = {
    "generic_name": "first",
    "brand_latin_name": "first",
    "count_of_package_sold": "sum",
    "sale_count": "sum",
    "sale_amount": "sum",
}
df = df.groupby(by="brand_latin_name", as_index=False).agg(agg_dict)
df["brand_latin_name"] = (
    df["brand_latin_name"].str.extract(r"^([A-Z\s]+?)\s{2,}")[0].str.strip()
)
df["dosage"] = df["generic_name"].str.extract(r"(\d+)\s*")[0].str.strip()
df["dosage"] = df["dosage"].astype(np.int64)

OUTPUT_PATH: Path = PROJECT_ROOT / "data" / "processed" / "factor_viii_2023.xlsx"
if OUTPUT_PATH.exists():
    override = input("File already exists, should override? (Y/N)")
    if override.lower() == "y":
        with pd.ExcelWriter(OUTPUT_PATH) as writer:
            print(f"Storing results at: {OUTPUT_PATH}")
            df_uniques.to_excel(writer, sheet_name="dataset", index=False)
            df.to_excel(writer, sheet_name="prices", index=False)
    else:
        print("Ignored")

In [ ]:
# Cleaning 2024 Summary statistics
df = pd.read_excel(
    io=PROJECT_ROOT / "data" / "raw" / "fda-statistics-summary-2024.xlsx"
).rename(columns={"فروش عددی 1403": "sale_count"})
df["dosage"] = df["generic_name"].str.extract(r"(\d+)\s*")[0].str.strip()
df["dosage"] = df["dosage"].astype(np.int64)

OUTPUT_PATH: Path = PROJECT_ROOT / "data" / "processed" / "factor_viii_2024.xlsx"
if OUTPUT_PATH.exists():
    override = input("File already exists, should override? (Y/N)")
    if override.lower() == "y":
        with pd.ExcelWriter(OUTPUT_PATH) as writer:
            print(f"Storing results at: {OUTPUT_PATH}")
            df.to_excel(writer, sheet_name="dataset", index=False)
    else:
        print("Ignored")
else:
    with pd.ExcelWriter(OUTPUT_PATH) as writer:
        print(f"Storing results at: {OUTPUT_PATH}")
        df.to_excel(writer, sheet_name="dataset", index=False)

Storing results at: /home/mohammad/projects/Thesis/hemophilia/data/processed/factor_viii_2024.xlsx
